In [1]:
from pkg.datasets import IDSDataset
from pkg.lit_ae import LitIDSAE

In [2]:
from torchvision import transforms, utils
from tqdm import tqdm
import pickle
import pandas as pd
# Маппинги айпи адресов
with open('Data/sip_map.pkl', 'rb') as f:
    sip_map = pickle.load(f)
with open('Data/dip_map.pkl', 'rb') as f:
    dip_map = pickle.load(f)

In [3]:
autoencoder = LitIDSAE(sip_map, dip_map, 128, 512, use_category=False)

In [4]:
# Use min max stats from whole dataset
ds_train = IDSDataset('Data/all_train.csv', sip_map, dip_map, only_benign=True, transform=True,
                     stat=pd.read_csv('Data/all_stat.csv').rename({0:'count', 1:'mean', 2:'std', 3:'min', 4:'25%', 5:'50%', 6:'75%', 7:'max'}))
#ds_test = IDSDataset('Data/all_test.csv', sip_map, dip_map, only_benign=True, transform=True)

In [5]:
from torch.utils.data import DataLoader
BATCH_SIZE = 3 ** 5
train_loader = DataLoader(ds_train, batch_size=BATCH_SIZE,
                        shuffle=True, num_workers=4)
# val_loader = DataLoader(ds_test, batch_size=BATCH_SIZE // 2,
#                         shuffle=False, num_workers=4)

In [6]:
import pytorch_lightning as pl

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger
logger = TensorBoardLogger(
    save_dir='exps',
#     version=1,
    name='autoencoder-final'
)
trainer = pl.Trainer(gpus=[1], logger=logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(autoencoder, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name               | Type       | Params
--------------------------------------------------
0 | continuous_emb     | Linear     | 8.7 K 
1 | encoder            | Sequential | 230 K 
2 | layer_mean         | Linear     | 16.4 K
3 | decoder            | Sequential | 246 K 
4 | continuous_decoder | Sequential | 8.6 K 
--------------------------------------------------
511 K     Trainable params
0         Non-trainable params
511 K     Total params
2.044     Total estimated model params size (MB)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=56316f98-13a2-452c-be31-8997c1e2f7a7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>